In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Classes provide a means of bundling data and functionality together. Creating a new class creates a new *type* of object, allowing new *instances* of that type to be made. Each class instance can have attributes attached to it for maintaining its state. Class instances can also have methods (defined by its class) for modifying it state.

Python classes provide all the standard features of Object Oriented Programming: the class inheritance mechanism allows multiple base classes, a derived class can override any methods of its base class or classes, and a method can call the method of a base class with the same name.

## A word about names and objects

Objects have individuality, and multiple names (in multiple scopes) can be bound to the same object. This is known as aliasing in other languags. However, aliasing has a possibly surprising effect on the semantics of Python code involving mutable objects, such as lists, dictionaries, and most other types.

## Python scopes and namespaces

A *namespace* is a mapping from names to objects. Most namespaces are currently implemented as Python dictionaries, but that's normally not noticeable in any way (except for performance), and it may change in the future. Examples of namespaces are:
- the set of built-in names (containing functions such as [abs](https://docs.python.org/3/library/functions.html#abs), and built-in exception names)
- the global names in a module
- the local names in a function invocation
- In a sense the set of attributes of an object also form a namespace.

Strictly speaking, references to names in modules are *attribute* reference: in the expression `modname.funcname`, `modname` is a module object and `funcname` is an attribute of it. In this case there happens to be a straightforward mapping between the module's attributes and the global names defined in the module: they share the same namespace!

Namespaces are created at different moments and have different lifetimes. The statements executed by the top-level invocation of the interpreter, either read from a script file or interactively, are considered part of a module called [__main__](https://docs.python.org/3/library/__main__.html#module-__main__), so they have their own global namespace. (The built-in names actually also live in **a** module; this is called [builtins](https://docs.python.org/3/library/builtins.html#module-builtins).)

A *scope* is a textual region of a Python program where a namespace is directly accessible. "Directly accessible" here means that an unqualified reference to a name attempts to find the name in the namespaces.

Although scopes are determined statically, they are used dynamically. At any time during execution, there are 3 or 4 nested scopes whose namespaces are directly accessible:
- the innermost scope, which is searched first, contains the local names
- the scopes of any enclosing functions, which are searched starting with the nearest enclosing scope, contain non-local, but also non-global names
- the next-to-last scope contains the current module's global names
- the outermost scope (searched last) is the namespace containing built-in names

If a name is declared global, then all references and assignments go directly to the next-to-last scope containing the module's global names. To rebind variables found outside of the innermost scope, the [nonlocal](https://docs.python.org/3/reference/simple_stmts.html#nonlocal) statement can be used; if not declared nonlocal, those variables are read-only (an attempt to write to such a variable will simply create a *new* local variable in the innermost scope, leaving the identically named outer variable unchanged).

The [global](https://docs.python.org/3/reference/simple_stmts.html#global) statement can be used to indicate that particular variables live in the global scope and should be rebound there; the [nonlocal](https://docs.python.org/3/reference/simple_stmts.html#nonlocal) statement indicates that particular variables live in an enclosing scope and should be rebound there.

In [2]:
# scopes and namespaces example
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


Note how the *local* assignment (which is default) didn't change *scope_test*'s binding of *spam*. The nonlocal assignment changed *scope_test*'s binding of *spam*, and the global assignment changed the module-level binding.

## A first look at classes

When a class definition is entered, a new **namespace** is created, and used as the **local scope** - thus, all assignments to local variables go into this new namespace. In particular, function definition bind the name of the *new* function here.

When a class definition is left normally (via the end), a *class object* is created. This is basically a wrapper around the contents of the namespace created by the class definition. The original local scope (the one in effect just before the class definition was entered) is reinstated, and the class object is bound here to the class name given in the class definition header.

Class objects support two kinds of operations: **attribute reference** and **instantiation**.

In [3]:
# attribute references
class MyClass:
    def __init__(self):
        self.data = []
        
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'


# dir() function returns MyClass's attributes, and recursively the attributes of its bases.
print(dir(MyClass))

# This attribute reference returns an integer
MyClass.i
# This attribute reference returns a function object
MyClass.f

# change the value of MyClass.i by assignment
MyClass.i = 123
MyClass.i

# __doc__ is a valid attribute, returns the docstring belonging to the class
MyClass.__doc__

# Class instantiation uses function notation, this returns a new instance of the class, and assigns this object to the local variable x
x = MyClass()
x
x.i
x.f
x.f()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'f', 'i']


12345

<function __main__.MyClass.f(self)>

123

123

<bound method MyClass.f of <__main__.MyClass object at 0x10abeeb90>>

'hello world'

Many classes like to create objects with instances customized to a specific *initial state*. Therefore a class may define a *special* method named \_\_init__(), like this:

In [4]:
def __init__(self):
    self.data = []

When a class defines an \_\_init__() method, class instantiation *automatically* invokes \_\_init__() for the newly created class instance.

In [5]:
x.data

[]

The \_\_init__() method may have *arguments* for greater flexibility. In that case, arguments given to the class instantiation operator are *passed to* \_\_init__(). For example:

In [6]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

y = Complex(3.0, -4.5)
y.r
y.i

3.0

-4.5

### Instance Objects

The only operations understood by instance objects are **attribute references**. There are two kinds of valid attribute names: *data attributes* and *methods*.

*Data attributes* need not be declared; like local variables, they spring into existence when they are first assigned to.

In [7]:
# This will print the value 16, without leaving a trace
x.counter = 1
while x.counter < 10:
    x.counter = x.counter * 2
print(x.counter)
del x.counter

16


The other kind of instance attribute reference is a *method*. A method is a function that "belongs to" an object.

Valid method names of an instance object depend on its class. By definition, all attributes of a class that are function objects define corresponding methods of its instances. So in above example, `x.f` is a valid method reference, since `MyClass.f` is a function; but `x.i` is not, since `MyClass.i` is not. But `x.f` is not the same thing as `MyClass.f` -- it is a *method object*, not a function object.

### Method Objects

In [8]:
# Usually, a method is called right after it is bound
x.f()

'hello world'

In [9]:
# However, it is not necessary to call a method right away; x.f is a method object and can be stored away
xf = x.f
x.counter = 1
while x.counter < 5:
    print(xf())
    x.counter = x.counter * 2

del x.counter

hello world
hello world
hello world


The special thing about methods is that the instance object is passed as the first argument of the function. In the above example, the call `x.f()` is exactly equivalent to `MyClass.f(x)`. In general, calling a method with a list of *n* arguments is equivalent to calling the corresponding function with an argument list that is created by inserting the method's instance object before the first argument.

like: `x.f(1, 2, 3)` is equivalent to `MyClass.f(x, 1, 2, 3)`

When a non-data attribute of an instance is referenced, the instance’s class is searched. If the name denotes a valid class attribute that is a function object, a **method object** is created by packing (pointers to) the *instance object* and the *function object* just found together in an abstract object: this is the method object. When the method object is called with an argument list, a new argument list is constructed from the instance object and the *argument list*, and the function object is called with this new argument list.

### Class and Instance Variables

Generally speaking, instance variables are for data *unique* to each instance and class variable are for attributes and methods *shared* by all instances of the class:

In [17]:
class Dog:

    kind = 'canine'    # class variable shared by all instances

    def __init__(self, name):
        self.name = name    # instance variable unique to each instance
        self.tricks = []    # creates a new empty list for each dog, it is also an instance variable
        
    def add_trick(self, trick):    # method shared by all instances
        self.tricks.append(trick)


d = Dog('Fido')
e = Dog('Buddy')

d.kind    # shared by all dogs
e.kind    # shared by all dogs

d.name    # unique to d
e.name    # unique to e

d.add_trick('roll over')
e.add_trick('play dead')
d.tricks
e.tricks

d.kind = 'cat'
d.kind

'canine'

'canine'

'Fido'

'Buddy'

['roll over']

['play dead']

'cat'

## Random Remarks

In [11]:
# if the same acctribute name occurs in both an instance and in a class, then attribute lookup prioritize the instance
class Warehouse:
    purpose = 'storage'
    region = 'west'

w1 = Warehouse()
print(w1.purpose, w1.region)

w2 = Warehouse
w2.region = 'east'
print(w2.purpose, w2.region)

storage west
storage east


In [13]:
# assigning a function object to a local variable in the class is also ok
# Function defining outside the class
def f1(self, x, y):
    return min(x, x+y)

class C:
    f = f1

    def g(self):
        return 'hello world'


Now, `f` and `g` are all attributes of class C that refer to function objects, and consequently, they are all methods of instances of C.

Methods may call other methods by using method attributes of the `self` argument:

In [14]:
class Bag:
    def __init__(self):
        self.data = []    # instnce variable unique to each instance

    def add(self, x):
        self.data.append(x)

    def addtwice(self, x):
        self.add(x)    # methods call other method
        self.add(x)

While one rarely encounters a good reason for using global data in a method. *Functions* and *modules* imported into the global scope can be used by methods, as well as function and *classes* defined in it.

Each value is an object, and therefore has a *class* (also called its *type*). It is stored as `object.__class__`.

## Inheritance

The syntax for a derived class definition looks like this:
```Python
class DerivedClassName(BaseClassName):
    <statement-1>
    .
    .
    .
    <statement-N>
```

When the base class is defined in another module:
```Python
class DerivedClassName(modname.BaseClassName)
```

An overriding method in a derived class may in fact want to *extend* rather than simply replace the base class method of the same name. There is a simple way to call the base class method directly: just call `BaseClassName.methodname(self, arguments)`.

Python has two built-in functions that work with inheritance:
- Use [isinstance()](https://docs.python.org/3/library/functions.html#isinstance) to check an instance’s type: `isinstance(obj, int)` will be True only if `obj.__class__` is int or some class *derived from* int.

- Use [issubclass()](https://docs.python.org/3/library/functions.html#issubclass) to check class inheritance: `issubclass(bool, int)` is `True` since [bool](https://docs.python.org/3/library/functions.html#bool) is a subclass of int. However, `issubclass(float, int)` is `False` since float is not a subclass of int.

a class definition with *multiple* base classes looks like this
```Python
class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <statement-N>
```

The method resolution order changes *dynamically* to support *cooperative calls* to [super()](https://docs.python.org/3/library/functions.html#super). 

*Dynamic ordering* is necessary because all cases of multiple inheritances exhibit one or more diamond relationships (where at least one of the parent classes can be accessed through multiple paths from the bottommost class).??

## Private Variables

There is a convention that is followed by most Python code: a name prefixed with an *underscore* (e.g. _spam) should be treated as a *non-public* part of the API (whether it is a function, a method or a data member). It should be considered an implementation detail and subject to change without notice.

In [26]:
# name mangling let subclass override methods without breaking intraclass method calls
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update    # private copy of original update() method

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __ini__()
        for item in zip(keys, values):
            self.items_list.append(item)

mapping = MappingSubclass([1, 2])
mapping.update.__self__    # the instance object with the method update()
mapping.update.__func__    # the function object corresponding to the method

<function __main__.MappingSubclass.update(self, keys, values)>

The above example would work even if `MappingSubclass` were to introduce a `__update` identifier since it is replaced with `_Mapping__update` in the Mapping class and `_MappingSubclass__update` in the `MappingSubclass` class respectively.

## Odds and Ends

use [dataclasses](https://docs.python.org/3/library/dataclasses.html#module-dataclasses) for bundling together a few named data items.

In [20]:
from dataclasses import dataclass

@dataclass
class Employee:
    name: str
    dept: str
    salary: int

In [21]:
john = Employee('John', 'computer lab', 1000)

john.dept
john.salary

'computer lab'

1000

A piece of Python code that expects a particular *abstract data type* can often be passed a class that *emulates* the methods of that data type instead. For instance, if you have a function that formats some data from a file object, you can define a class with methods read() and readline() that get the data from a string buffer instead, and pass it as an argument. ??

Instance method objects have *attributes*, too: m.\_\_self__ is the *instance object* with the method m(), and m.\_\_func__ is the *function object* corresponding to the method.

## Iterators

In [37]:
# most container objects can be looped over using a for statement
for ele in 'String':
    print(ele)

for ele in [1, 2, 3]:
    print(ele)

for ele in (1, 2, 3):
    print(ele)

for key in {'one': 1, 'two': 2}:
    print(key)

print(ele, key)

S
t
r
i
n
g
1
2
3
1
2
3
one
two
3 two


This style of access is clear, concise, and convenient. The use of iterators *pervades* and *unifies* Python. Behind the scenes, the [for](https://docs.python.org/3/reference/compound_stmts.html#for) statement calls [iter()](https://docs.python.org/3/library/functions.html#iter) on the container object. The function returns an *iterator object* that defines the method [\_\_next__](https://docs.python.org/3/library/stdtypes.html#iterator.__next__) which accesses elements in the container one at a time. When there are no more elements, `__next__()` raises a StopIteration exception which tells the for loop to terminate. You can call the \_\_next__() method using the [next()](https://docs.python.org/3/library/functions.html#next) built-in function;

In [40]:
s = 'abc'
it = iter(s)
it

it.__next__

next(it)
next(it)
next(it)

<method-wrapper '__next__' of str_ascii_iterator object at 0x10bd27be0>

'a'

'b'

'c'

Having seen the mechanics behind the *iterator protocol*, it is easy to add iterator behavior to your classes.

Define an \_\_iter__() method which returns an object with a \_\_next__() method. If the class defines \_\_next__(), then \_\_iter__() can just return self:

In [51]:
class Reverse:
    """Iterator for looping over a sequence backwards"""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [67]:
rev = Reverse('spam')
rev
iter(rev)


for char in rev:
    print(char)

m
a
p
s


## Generator

In [66]:
def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]

for char in reverse('golf'):
    print(char)

reverse('golf')

f
l
o
g


<generator object reverse at 0x10b895a80>

Anything that can be done with generators can also be done with *class-based* iterators. What makes generators so compact is that the \_\_iter__() and \_\_next__() methods are created *automatically*.

Another key feature is that the *local variables* and *execution state* are automatically *saved* between calls. This made the function easier to write and much more clear than an approach using instance variables like `self.index` and `self.data`.

## Generator Expressions

Some simple generators can be coded succinctly as expressions using a syntax similar to list comprehensions but with *parentheses* instead of square brackets. These expressions are designed for situations where the generator is used *right away* by an enclosing function. 

*Generator expressions* are more compact but less versatile than full generator definitions and tend to be more *memory friendly* than equivalent list comprehensions.

Examples:

In [68]:
sum(i*i for i in range(10))    # sum of squares

xvec = [10, 20, 30]
yvec = [7, 5, 3]
sum(x*y for x,y in zip(xvec, yvec))    # dot product

data = 'golf'
list(data[i] for i in range(len(data)-1, -1, -1))

285

260

['f', 'l', 'o', 'g']